# **Data Importation**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Main**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import re
from sklearn.preprocessing import LabelEncoder

In [ ]:
tr = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022//train.csv')
ts = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022//test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022//sample_submission.csv')

In [ ]:
tr.head(3)

In [ ]:
print(tr.shape)
print(ts.shape)

In [ ]:
sns.countplot(tr['country'])

In [ ]:
sns.distplot(tr['num_sold'])

# **Feature engineering**

In [ ]:
df = pd.concat([tr,ts])

In [ ]:
df['day'] = pd.to_datetime(df.date).dt.day
df['day_w'] = pd.to_datetime(df.date).dt.dayofweek
df['month'] = pd.to_datetime(df.date).dt.month
df['year'] = pd.to_datetime(df.date).dt.year

In [ ]:
def trait_season(x):
  if ((x<=4)&(x>=2)):
    return 0
  elif ((x<=7)&(x>=5)):
    return 1
  elif ((x<=10)&(x>=8)):
    return 2
  else:
    return 3
df['season'] = df['month'].apply(trait_season)

In [ ]:
def trait_weekend(x):
  if x>=5:
    return 1
  else :
    return 0
df['weekend'] = df['day_w'].apply(trait_weekend)

In [ ]:
#Public holiday and school holidays

In [ ]:
!pip install holidays

In [ ]:
from datetime import date
import holidays

In [ ]:
fin_holidays = holidays.FIN()
nor_holidays = holidays.NOR()
swd_holidays = holidays.SWE()

In [ ]:
holidays=[]
for k in range(len(df)):
  if df['country'].iloc[k] == 'Finland':
    if df['date'].iloc[k] in fin_holidays:
      holidays.append(1)
    else:
      holidays.append(0)
  elif df['country'].iloc[k] == 'Norway':
    if df['date'].iloc[k] in nor_holidays:
      holidays.append(1)
    else:
      holidays.append(0)
  else:
    if df['date'].iloc[k] in swd_holidays:
      holidays.append(1)
    else:
      holidays.append(0)
df['holidays'] = holidays

In [ ]:
df.groupby('country')['store'].value_counts()

In [ ]:
sns.countplot(tr['product'])

In [ ]:
#PIB
pib_fin = {"2015":234.4,"2016":240.6,"2017":255,"2018":275.8,"2019":269}
pib_nor = {"2015":385.5,"2016":368.8,"2017":398.4,"2018":437,"2019":405.5}
pib_swd = {"2015":505.1,"2016":515.7,"2017":541,"2018":555.5,"2019":531.3}

In [ ]:
#https://www.journaldunet.com/business/salaire/classement/pays/revenus
rev_fin = {"2015":3877.5,"2016":3754.2,"2017":3732.5,"2018":4019.2,"2019":4163.3}
rev_nor = {"2015":7119.2,"2016":6865.8,"2017":6304.2,"2018":6693.3,"2019":6801.7}
rev_swd = {"2015":4813.3,"2016":4540,"2017":4419.2,"2018":4636.7,"2019":4674.2}

In [ ]:
#Inflation
inf_fin = {"2015":-0.16,"2016":0.39,"2017":0.84,"2018":1.17,"2019":1.33}
inf_nor = {"2015":2.17,"2016":3.55,"2017":1.88,"2018":2.77,"2019":2.17}
inf_swd = {"2015":0.05,"2016":1.74,"2017":1.74,"2018":2.04,"2019":1.75}

In [ ]:
#unemployment
un_fin = {"2015":9.38,"2016":8.79,"2017":8.7,"2018":8.11,"2019":7.78}
un_nor = {"2015":4.37,"2016":4.7,"2017":4,"2018":3.8,"2019":3.7}
un_swd = {"2015":7.4,"2016":6.95,"2017":6.57,"2018":6.33,"2019":6.3}

In [ ]:
pib_fin[str(df['year'].iloc[0])]

In [ ]:
pib=[]
rev=[]
inf=[]
unp=[]
for k in range(len(df)):
  if df['country'].iloc[k] == 'Finland':
    pib.append(pib_fin[str(df['year'].iloc[k])])
    rev.append(rev_fin[str(df['year'].iloc[k])])
    inf.append(inf_fin[str(df['year'].iloc[k])])
    unp.append(un_fin[str(df['year'].iloc[k])])
  elif df['country'].iloc[k] == 'Norway':
    pib.append(pib_nor[str(df['year'].iloc[k])])
    rev.append(rev_nor[str(df['year'].iloc[k])])
    inf.append(inf_nor[str(df['year'].iloc[k])])
    unp.append(un_nor[str(df['year'].iloc[k])])
  else:
    pib.append(pib_swd[str(df['year'].iloc[k])])
    rev.append(rev_swd[str(df['year'].iloc[k])])
    inf.append(inf_swd[str(df['year'].iloc[k])])
    unp.append(un_swd[str(df['year'].iloc[k])])

In [ ]:
df['PIB'] = pib
df['REV'] = rev
df['INF'] = inf
df['UNP']= unp

In [ ]:
df.head(2)

In [ ]:
df.drop(['row_id','date'],1,inplace=True)

In [ ]:
print(tr.shape)

In [ ]:
cols=['country','store','product']
le=LabelEncoder()
for col in cols:
  df[col] = le.fit_transform(df[col])

In [ ]:
tr_data = df[:26298]
ts_data = df[26298::]
ts_data.drop('num_sold',1,inplace=True)
print(tr_data.shape)

In [ ]:
colormap = sns.diverging_palette(220, 10, as_cmap = True)
plt.figure(figsize=(15,15))
sns.heatmap(tr_data.corr(), cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 12},cmap = colormap, linewidths=0.1, linecolor='white')

# **Modelling**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = tr_data.drop('num_sold',1)
Y = tr_data['num_sold']

In [ ]:
x_tr,x_val,y_tr,y_val=train_test_split(X,Y,test_size=0.15)

In [ ]:
def smape_loss(y_true, y_pred):
    """SMAPE Loss"""
    return np.abs(y_true - y_pred) / (y_true + np.abs(y_pred)) * 200

# **Feature selection**

In [ ]:
#import statsmodels.api as sm
#from scipy import stats

In [ ]:
#X2 = sm.add_constant(X)
#est = sm.OLS(Y, X2)
#est2 = est.fit()
#print(est2.summary())

In [ ]:
# Great !!, All p_value are < 5%

In [ ]:
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import f_regression

In [ ]:
#fs = SelectKBest(score_func=f_regression, k=13)
#x_sb= fs.fit_transform(X,Y)

In [ ]:
#display the selected features
#l_f = fs.get_support()
#l_f2 = ts_data.columns
#l_f2 = list(l_f2)

#fs_se = []
#for i in range (len(l_f2)):
#  if l_f[i] == True :
#    fs_se.append(l_f2[i])
    
#new_test = ts_data[fs_se]

In [ ]:
#x_tr1,x_val1,y_tr1,y_val1=train_test_split(x_sb,Y,test_size=0.15)

# **LightGBM**

In [ ]:
#Try with log
y_tr_lg = y_tr.apply(np.log)
y_val_lg = y_val.apply(np.log)

In [ ]:
import lightgbm as lgb
trn_data_lg = lgb.Dataset(x_tr, label=y_tr_lg)
val_data_lg = lgb.Dataset(x_val, label=y_val_lg)

In [ ]:
#LGB with params0

In [ ]:
params0 = {'objective': 'regression', # Manual optimization
           'force_row_wise': True,
           'max_bin': 400, # need more bins than days in a year
           'verbosity': -1,
           'seed': 1,
           'bagging_seed': 3,
           'feature_fraction_seed': 2,
           'learning_rate': 0.018,
           'lambda_l1': 0,
           'lambda_l2': 1e-2,
           'num_leaves': 10,
           'feature_fraction': 0.710344827586207,
           'bagging_fraction': 0.47931034482758617,
           'bagging_freq': 5,
           'min_child_samples': 20}

In [ ]:
lgb_model_lg = lgb.train(params0, trn_data_lg, 25000, valid_sets = [trn_data_lg, val_data_lg], verbose_eval=2000)

In [ ]:
pred_val_lg = lgb_model_lg.predict(x_val,num_iteration=lgb_model_lg.best_iteration)

In [ ]:
pred_val2 = np.exp(pred_val_lg)

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(y_val, pred_val2, s=1, color='r')
plt.xlabel('y_true')
plt.ylabel('y_pred')
plt.show()

In [ ]:
smape = np.mean(smape_loss(y_val, pred_val2))
smape

**Test prediction&submission**

In [ ]:
pred_test_lg = lgb_model_lg.predict(ts_data,num_iteration=lgb_model_lg.best_iteration)

In [ ]:
pred_test = np.exp(pred_test_lg)

In [ ]:
sub['num_sold'] = pred_test

In [ ]:
sub.to_csv('third.csv',index=False)

# **GridSearchCV**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
params_gs = {'objective': ['regression'], # Manual optimization
           'force_row_wise': [True],
           'max_bin': [350,400], # need more bins than days in a year
           'learning_rate': [0.01,0.05,0.1],
           'lambda_l2': [1e-1,1e-2,1e-3],
           'num_leaves': [10,20,30],
           'max_depth': [10,15,20],
           'feature_fraction': [0.85],
           'bagging_fraction': [0.8],
           'bagging_freq':[3],
           'min_child_samples': [10,20,30]}

In [ ]:
lgb_model_lg = lgb.LGBMRegressor()

In [ ]:
gsearch_lgb = GridSearchCV(lgb_model_lg, param_grid = params_gs, verbose=1)

In [ ]:
gsearch_lgb.fit(x_tr,y_tr_lg)

In [ ]:
print('best params')
print (gsearch_lgb.best_params_)

In [ ]:
best_lg = {'bagging_fraction': 0.8, 'bagging_freq': 3, 'feature_fraction': 0.85, 'force_row_wise': True, 'lambda_l2': 0.1, 'learning_rate': 0.1, 'max_bin': 350, 'max_depth': 15, 'min_child_samples': 10, 'num_leaves': 30, 'objective': 'regression'}

In [ ]:
#Training with best parameters
lgb_model_lg = lgb.train(best_lg, trn_data_lg, 18000, valid_sets = [trn_data_lg, val_data_lg], verbose_eval=2000)

In [ ]:
pred_val_lg = lgb_model_lg.predict(x_val,num_iteration=lgb_model_lg.best_iteration)

In [ ]:
pred_val3 = np.exp(pred_val_lg)

In [ ]:
smape = np.mean(smape_loss(y_val, pred_val3))
smape

In [ ]:
preds_test = lgb_model_lg.predict(ts_data)

In [ ]:
sub['num_sold'] = np.exp(preds_test)
sub.to_csv('best.csv',index=False)

# **CatBoostRegressor**

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cat_model = CatBoostRegressor(iterations= 25000,
        depth= 7, 
        learning_rate=0.02,
        l2_leaf_reg= 12.06,
        bootstrap_type= 'Bayesian',
        boosting_type= 'Plain',
        loss_function= 'MAPE',
        eval_metric= 'SMAPE',
        #od_type= 'Iter',       # type of overfitting detector
        #od_wait= 40,
        has_time= True)

In [ ]:
cat_model.fit(x_tr, y_tr_lg,
             eval_set=(x_val, y_val_lg),
             use_best_model=True,
             verbose=2000)

In [ ]:
pred_val_xg = cat_model.predict(x_val)

In [ ]:
pred_val_xg = np.exp(pred_val_xg)
print('SMAPE : ')
np.mean(smape_loss(y_val, pred_val_xg))

**Test prediction&submission**

In [ ]:
pred_test = cat_model.predict(ts_data)
pred_test = np.exp(pred_test)

In [ ]:
sub['num_sold']=pred_test
sub.to_csv('cat.csv',index=False)

# **XGBoost**

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgbmodel = XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.07, max_depth=12, 
                             min_child_weight=1.7817, n_estimators=5000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
xgbmodel.fit(x_tr,y_tr_lg)

In [ ]:
pred_xgb = xgbmodel.predict(x_val)
pred_xgb = np.exp(pred_xgb)

In [ ]:
print('SMAPE : ')
np.mean(smape_loss(y_val, pred_xgb))

# **Stacking**

- This model (stacking) works great and I'm running it on colab but since it takes about 15 minutes to run I'm not running it here on Kaggle.

In [ ]:
from sklearn.ensemble import StackingRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [ ]:
params0 = {'objective': 'regression', # Manual optimization
           'force_row_wise': True,
           'max_bin': 400, # need more bins than days in a year
           'verbosity': -1,
           'seed': 1,
           'bagging_seed': 3,
           'feature_fraction_seed': 2,
           'learning_rate': 0.018,
           'lambda_l1': 0,
           'lambda_l2': 1e-2,
           'num_leaves': 10,
           'feature_fraction': 0.710344827586207,
           'bagging_fraction': 0.47931034482758617,
           'bagging_freq': 5,
           'min_child_samples': 20}

In [ ]:
#Try with log
y_tr_lg = y_tr.apply(np.log)
y_val_lg = y_val.apply(np.log)

In [ ]:
estimators = [
     ('XGB',XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.07, max_depth=12, 
                             min_child_weight=1.7817, n_estimators=5000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1) ),
     ('CAT',CatBoostRegressor(iterations= 25000,
        depth= 7, 
        learning_rate=0.02,
        l2_leaf_reg= 12.06,
        bootstrap_type= 'Bayesian',
        loss_function= 'MAPE',
        eval_metric= 'SMAPE',
        has_time= True))]

In [ ]:
reg = StackingRegressor(
     estimators=estimators,
     final_estimator=lgb.LGBMRegressor(**params0),verbose=100)

In [ ]:
reg.fit(x_tr, y_tr_lg)

In [ ]:
pred_st = reg.predict(x_val)
pred_st = np.exp(pred_st)

In [ ]:
np.mean(smape_loss(y_val, pred_st))

In [ ]:
pred_test = reg.predict(ts_data)
pred_test = np.exp(pred_test)

In [ ]:
sub['num_sold'] = pred_test

In [ ]:
sub.to_csv('stack.csv',index=False)